## Notebook to display boreholes on a map and to create the gpkg database from an existing sql3database and shapefile data

### Copy tables of a SQL3 database to another database

In [1]:
import geopandas as gpd
import pandas as pd
import folium
import sqlite3

In [2]:
def db_copy_sqlite3_tables(db_in, db_out, tables):
    """
    Copy tables from one database to another
    :param db_in: full filename of the origin of tables 
    :type db_in: string
    :param db_out: full filename of the destination of tables
    :type db_out: string
    :param tables: list of tables to copy
    :type tables: list
    :return: status, 0: OK, 1: failure
    :rtype: int
    """
    import sqlite3
    import pandas as pd
    
    status = 0
    try:
        conn_in = sqlite3.connect(db_in)
        conn_out = sqlite3.connect(db_out)
        for i in tables:
            df = pd.read_sql_query("SELECT * from {table:s}".format(table=i), conn_in)
            df.to_sql("{table:s}".format(table=i),conn_out, if_exists='replace', index = False)
        conn_in.close()
        conn_out.close()
    except Exception as e:
        print('Error copying tables:\n {error:s}'.format(error=e))
        status = 1
    return status    

### Access SQlite

In [3]:
conn = sqlite3.connect('../project_database.db')
conn
cursor = conn.cursor()

### List of tables

In [4]:
tables = []
for table in cursor.execute("SELECT name FROM sqlite_master WHERE type='table';"):
    print(table)
    tables.append(table)

('Boreholes',)
('Intervals',)
('Components',)
('Lexicon',)


In [5]:
list_tables = ['Components', 'Intervals', 'Boreholes','Lexicon']

## Create GeoDataFrame from shp 

Read shp file (as a geodataframe)

In [6]:
dat_shp = gpd.read_file("../playground/pilote/pilote/Pilote_v7.0.shp")

Read GPKG as a GeoDataFrame

Write a geojson from gdf

In [16]:
#data_from_gpkg.to_file("data.geojson", driver='GeoJSON')

## CRS

EPSG 31370 (Lambert 72)

In [7]:
dat_84 = dat_shp.to_crs(epsg=4326)

### Create GPKG from gdf

In [8]:
dat_84.to_file("dat.gpkg", driver="GPKG")

### Copy Tables

In [9]:
db_copy_sqlite3_tables('project_database.db','dat.gpkg',list_tables)

0

## Folium 

In [10]:
dat_gdf = gpd.read_file("dat.gpkg")

In [11]:
dat_84.to_file("output.json", driver="GeoJSON")

In [12]:
ice_map = folium.Map(location=[50.455, 3.94],
                    zoom_start=10)
folium.GeoJson(
    'output.json').add_to(ice_map)
ice_map

save in a file

In [13]:
ice_map.save('index.html')

### Add an Id column with the good identifyer

In [14]:
dat_gdf['id'] = 'F'+data_from_gpkg['Ref']

NameError: name 'data_from_gpkg' is not defined